<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# # !sudo apt-get update -y
# # !sudo apt-get install python3.8
# # from IPython.display import clear_output 
# # clear_output()
# # !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
# # !sudo update-alternatives --config python3
# # !python3 --version
# !sudo apt install python3-pip





In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
!pip install pyLDAvis

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa


from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip._internal'


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [38]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [39]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(5)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2002 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2010 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchu

In [40]:
bug_report["Priority"].value_counts()

P3    74830
P2     5108
P1     2077
P4     1942
P5     1199
Name: Priority, dtype: int64

In [41]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 85.6 ms, sys: 4.21 ms, total: 89.8 ms
Wall time: 96.4 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [42]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 481 ms, sys: 7.83 ms, total: 489 ms
Wall time: 493 ms


In [43]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 15.2 s, sys: 69.3 ms, total: 15.2 s
Wall time: 15.3 s


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [44]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

CPU times: user 25.4 s, sys: 445 ms, total: 25.8 s
Wall time: 27.1 s


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [45]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 9.77 ms, sys: 2.12 ms, total: 11.9 ms
Wall time: 13 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,25,P3,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [46]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

CPU times: user 23.3 ms, sys: 899 µs, total: 24.2 ms
Wall time: 24.7 ms


,Issue_id,Priority,Duplicated_issue,Title,Description
0,1,P3,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [47]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [48]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

CPU times: user 2.44 s, sys: 129 ms, total: 2.57 s
Wall time: 2.54 s


[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1),
 (4543, 2),
 (4544, 1),
 (4545, 1),
 (4546, 1),
 (4547, 1)]

In [49]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. "actually" word seen 1 time.
2. "builders" word seen 2 time.
3. "change" word seen 5 time.
4. "comment" word seen 1 time.
5. "compare" word seen 1 time.
6. "complete" word seen 1 time.
7. "consider" word seen 1 time.
8. "consistency" word seen 1 time.
9. "contain" word seen 1 time.
10. "default" word seen 1 time.
11. "derive" word seen 1 time.
12. "document" word seen 1 time.
13. "drive" word seen 1 time.
14. "editor" word seen 3 time.
15. "effect" word seen 1 time.
16. "ensure" word seen 1 time.
17. "external" word seen 3 time.
18. "extremely" word seen 1 time.
19. "fail" word seen 1 time.
20. "inconsistent" word seen 1 time.
21. "internal" word seen 1 time.
22. "maintain" word seen 2 time.
23. "mandatory" word seen 1 time.
24. "miss" word seen 1 time.
25. "modify" word seen 1 time.
26. "move" word seen 1 time.
27. "navigator" word seen 2 time.
28. "network" word seen 1 time.
29. "occur" word seen 1 time.
30. "open" word seen 1 time.
31. "optimal" word seen 1 time.
32. "problem" wo

In [50]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

[(0,
  '0.037*"editor" + 0.021*"eclipse" + 0.020*"window" + 0.019*"reproduce" + '
  '0.018*"select" + 0.018*"create" + 0.018*"button" + 0.014*"windows" + '
  '0.013*"dialog" + 0.013*"attachment"'),
 (1,
  '0.066*"message" + 0.061*"method" + 0.031*"exception" + 0.021*"bundle" + '
  '0.020*"javathread" + 0.019*"occur" + 0.018*"eclipse" + '
  '0.017*"threadblocked" + 0.015*"fail" + 0.015*"orgeclipseosgi"'),
 (2,
  '0.056*"warn" + 0.051*"target" + 0.036*"property" + 0.022*"properties" + '
  '0.021*"variable" + 0.015*"variables" + 0.014*"script" + 0.013*"reference" + '
  '0.009*"archive" + 0.008*"include"'),
 (3,
  '0.049*"eclipse" + 0.025*"update" + 0.023*"plugin" + 0.023*"feature" + '
  '0.022*"version" + 0.013*"platform" + 0.011*"require" + 0.011*"install" + '
  '0.011*"instal" + 0.010*"plugins"'),
 (4,
  '0.093*"thread" + 0.047*"daemon" + 0.047*"method" + 0.029*"wait" + '
  '0.029*"native" + 0.027*"lock" + 0.025*"monitor" + 0.024*"available" + '
  '0.022*"runnable" + 0.021*"worker"'),
 

In [51]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.36161169562314

Coherence Score:  0.6030106118674695
CPU times: user 37.5 s, sys: 532 ms, total: 38 s
Wall time: 37.8 s


In [52]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

ERROR:concurrent.futures:exception calling callback for <Future at 0x7f8e3f175c10 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 794, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.7/dist-packages/joblib/_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func)

TerminatedWorkerError: ignored

In [53]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."


bow_vector = topic=lda_model[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Score: 0.5438761115074158	 Topic: 0.031*"return" + 0.023*"method" + 0.020*"search" + 0.019*"object" + 0.013*"change" + 0.013*"call" + 0.012*"create" + 0.011*"string" + 0.011*"result" + 0.009*"implement"
Score: 0.05069471895694733	 Topic: 0.049*"eclipse" + 0.025*"update" + 0.023*"plugin" + 0.023*"feature" + 0.022*"version" + 0.013*"platform" + 0.011*"require" + 0.011*"install" + 0.011*"instal" + 0.010*"plugins"
Score: 0.05068648234009743	 Topic: 0.072*"project" + 0.027*"select" + 0.026*"create" + 0.022*"workspace" + 0.022*"change" + 0.022*"dialog" + 0.015*"eclipse" + 0.013*"resource" + 0.013*"launch" + 0.012*"delete"
Score: 0.050682395696640015	 Topic: 0.037*"editor" + 0.021*"eclipse" + 0.020*"window" + 0.019*"reproduce" + 0.018*"select" + 0.018*"create" + 0.018*"button" + 0.014*"windows" + 0.013*"dialog" + 0.013*"attachment"
Score: 0.050680771470069885	 Topic: 0.139*"source" + 0.043*"action" + 0.030*"extension" + 0.025*"command" + 0.015*"attribute" + 0.015*"bind" + 0.013*"context" + 0.

In [54]:
for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("drive/My Drive/duplicate_detection/topic_{}.csv")'.format(c,c))

FileNotFoundError: ignored

In [79]:
topic=lda_model[dictionary.doc2bow(master_reports.Description[0])]
print(topic)
topic= np.asarray(topic)
# print(topic)
print(topic[:,1])
print(np.argmax(topic[:,1]),0)
print(int(topic[np.argmax(topic[:,1]),0]))

[(0, 0.06832481), (3, 0.25818813), (4, 0.024682634), (6, 0.64206207)]
[0.06832481 0.25818813 0.02468263 0.64206207]
3 0
6


6
